# WNTR / EPANET-MSX demo
This demo shows a simple example of multispecies chlorine decay taken from the 
EPANETMSX user manual. The Net3 example network from EPANET is used, and two 
different decay coefficients are assigned - one for each source of water.
The river uses decay coefficient k1, the lake uses decay coefficient k2, and 
the two values are an order of magnitude different. Once the initial example,
from the EPANETMSX user manual, has been run, parameter sensitivity is performed
to look at the impacts of different decay coefficients for the river source.

In [2]:
from pprint import pprint

### Load the network model, optionally remove EPANET quality

In [3]:
import wntr


wn = wntr.network.WaterNetworkModel("../networks/Net3.inp")

wn.options.quality.parameter = "NONE"

### Add a new MSX model to the water network, set options

In [4]:
wn.add_msx_model()
wn.msx.title = "Multisource Chlorine Decay"
wn.msx.references.append(
    """(2023) Shang F, L Rossman, and J Uber. 
"EPANET-MSX 2.0 User Manual". EPA/600/R-22/199"""
)
wn.msx.options.area_units = "FT2"
wn.msx.options.rate_units = "DAY"
wn.msx.options.timestep = 300

### Add the MSX reaction system information
This reaction system comes from the EPANET-MSX user manual. There are two species: Free Chlorine and a tracer. The tracer is used to select which decay coefficient is being used. The river is source 1, the lake is source 2.

The amount of free chlorine is based on the rate reaction:

$$
    \frac{d}{dt}\mathrm{Cl_2} = -(k_1 T_1 + k_2(1-T_1)) \mathrm{Cl_2}
$$

In [ ]:
T1 = wn.msx.add_species("T1", "bulk", units="MG", note="Source 1 Tracer")
Cl2 = wn.msx.add_species("CL2", "bulk", units="MG", note="Free Chlorine")
print(repr(Cl2))

In [ ]:
k1 = wn.msx.add_constant("k1", 1.3, units="1/day")
k2 = wn.msx.add_constant("k2", 17.7, units="1/day")
print(repr(k2))

In [ ]:
rxn_T1 = wn.msx.add_reaction("T1", "pipe", "rate", "0")
rxn_Cl2 = wn.msx.add_reaction("CL2", "pipe", "rate", "-(k1*T1 + k2*(1-T1))*CL2")
print(repr(rxn_Cl2))

### Set up the initial quality
In this example, the initial quality is based on the two sources: the tracer, indicating which source is which, is set to 1.0 for the river; the chlorine is being boosted at the sources to the same level, 1.2 mg/L.

In [ ]:
from wntr.msx.elements import InitialQuality

net_data = wn.msx.network_data
net_data.initial_quality["T1"] = InitialQuality(node_values={"River": 1.0})
net_data.initial_quality["CL2"] = InitialQuality(node_values={"River": 1.2, "Lake": 1.2})
pprint(net_data.initial_quality)

### Run the simulation and view the results
With the MSX model attached to the WaterNetworkModel, there is nothing different in how the EpanetSimulator is called. Results are saved in keys with the species' name.

In [ ]:
sim = wntr.sim.EpanetSimulator(wn)
res = sim.run_sim()
print("Node results:", ", ".join([k for k in res.node.keys()]))
print("Link results:", ", ".join([k for k in res.link.keys()]))

Generate some graphics that show how the river fraction changes through time.

In [ ]:
import matplotlib.pyplot as plt

_ = wntr.graphics.plot_network(
    wn,
    node_attribute=res.node["T1"].loc[3600 * 12, :],
    title="12 h",
    node_colorbar_label="River\nFraction",
)
_ = wntr.graphics.plot_network(
    wn,
    node_attribute=res.node["T1"].loc[3600 * 24, :],
    title="24 h",
    node_colorbar_label="River\nFraction",
)
_ = wntr.graphics.plot_network(
    wn,
    node_attribute=res.node["T1"].loc[3600 * 36, :],
    title="36 h",
    node_colorbar_label="River\nFraction",
)
query = "117"  # '191', '269', '117'
res.node["CL2"][query].plot()
res.node["T1"][query].plot()
plt.title("Node {}\nk1 = {:.1f}, k2 = {:.1f}".format(query, k1.value, k2.value))
_ = plt.legend(["Cl2", "T1"])

### Look at impact of different k1 values on residuals

In [11]:
d_k1 = dict()
k1 = wn.msx.reaction_system.constants["k1"]
for i in range(7):
    # Increase the reaction rate
    newk = 1.3 + i * 2.6
    k1.value = newk
    resk = sim.run_sim()
    d_k1[newk] = resk

In [ ]:
# res.node["T1"].loc[0:3600*36, query].plot(style='k.')
for newk, resk in d_k1.items():
    resk.node["CL2"].loc[0 : 3600 * 36, query].plot()
plt.legend(["{:.1f}".format(k) for k in d_k1.keys()], title="k1 (1/day)")
plt.title("Chlorine residual at node {}".format(query))
plt.xlabel("Seconds")
plt.ylabel("Concentraion [mg/L]")

### Save the model
The model is now saved in two formats: the EPANET-MSX style format as Net3.msx and then as a JSON file, Net3-msx.json.
We also can save the model as in a library format; this strips the JSON file of any network-specific information so that it only contains the species, constants, and reaction dynamics.

In [13]:
from wntr.msx import io as msxio

msxio.write_msxfile(wn.msx, "Net3.msx")
msxio.write_json(wn.msx, "Net3-msx.json")
msxio.write_json(wn.msx, "multisource-cl.json", as_library=True)

We can look at the file that was written out, and we can read in the two JSON files to see how the library format has stripped out the network data.

In [ ]:
with open("Net3.msx", "r") as fin:
    print(fin.read())

In [ ]:
import json

with_net: dict = None
without_net: dict = None

with open("Net3-msx.json", "r") as fin:
    with_net = json.load(fin)
with open("multisource-cl.json", "r") as fin:
    without_net = json.load(fin)

print("With network data:")
pprint(with_net["network_data"])

print("As a library:")
pprint(without_net["network_data"])

### Using the MSX WNTR library
WNTR now includes a library functionality that allows a user to access certain objects by name.
The MSX integration includes adding a library of certain reaction models that are described in
the EPANET-MSX user manual. This section demonstrates how to use the model that was just saved
in the library.

In [ ]:
from wntr.library.msx import MsxLibrary

my_library = MsxLibrary(extra_paths=["."])  # load files from the current directory
my_library.model_name_list()

Note that this has pulled in a lot more files than might be expected. This is because it is getting the default models (the first five models) followed by all the models in the current directory ('.'). It grabs any file that has the extension .json or .msx, which means that it has pulled in "temp", since this demo has run the EpanetSimulator several times, Net3, Net3-msx, and multisource-cl, because they were just created.

The models are accessed by name. To see how they are different, compare the initial quality for the "Net3" model (which came from the .msx file created above) and the "multisource-cl" model (created with as_library=True).

In [ ]:
my_library.get_model("Net3").network_data.initial_quality

In [ ]:
my_library.get_model("multisource-cl").network_data.initial_quality

Now, examine a model that comes from the built-in data.

In [ ]:
arsenic = my_library.get_model("arsenic_chloramine")
for key, value in arsenic.reaction_system.variables():
    print(repr(value))
for key, value in arsenic.reaction_system.reactions():
    print(repr(value))